In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import os

Primero, se busca calcular la matriz de distancias eucledianas entre zonas y tiendas.

In [31]:
path_zonas = os.path.join('..', '..', 'Datos', 'zonas_20250115.csv')
path_tiendas = os.path.join('..', '..', 'Datos', 'tiendas_20250115.csv')
zonas_data = pd.read_csv(path_zonas)
tiendas_data = pd.read_csv(path_tiendas)

coords_zonas = zonas_data[['x_zona', 'y_zona']].values
coords_tiendas = tiendas_data[['pos_x', 'pos_y']].values
# Se resta 1 a las coordenadas de las tiendas para que coincidan con el sistema de coordenadas de las zonas
coords_tiendas[:, 0] -= 1
coords_tiendas[:, 1] -= 1

#Se calcula la matriz de distancias entre zonas y tiendas
dist_matrix = cdist(coords_zonas, coords_tiendas, metric='euclidean')

#Se convierte a df para facilitar lectura
dist_df = pd.DataFrame(
    dist_matrix,
    index=zonas_data['id_zona'],
    columns=tiendas_data['id_tienda']
)

print(dist_df.head())

id_tienda         1          2          3          4          5         6   \
id_zona                                                                      
1          23.259407  54.708317  49.244289  62.177166  55.731499  3.162278   
2          22.360680  54.203321  48.846699  61.717096  55.578773  2.236068   
3          21.470911  53.712196  48.466483  61.269895  55.443665  1.414214   
4          20.591260  53.235327  48.104054  60.835845  55.326305  1.000000   
5          19.723083  52.773099  47.759816  60.415230  55.226805  1.414214   

id_tienda         7          8          9          10         11         12  \
id_zona                                                                       
1          29.410882  46.173586  20.591260  55.081757  28.792360  45.453273   
2          28.600699  46.097722  19.723083  54.817880  28.460499  44.821870   
3          27.802878  46.043458  18.867962  54.571055  28.160256  44.204072   
4          27.018512  46.010868  18.027756  54.341513  27.

Luego, la matríz de distancias eucledianas entre solo zonas.

In [7]:
dist_matrix = cdist(coords_zonas, coords_zonas, metric='euclidean')

dist_df = pd.DataFrame(
    dist_matrix,
    index=zonas_data['id_zona'],
    columns=zonas_data['id_zona']
)

print(dist_df.head())

id_zona  1     2     3     4     5     6     7     8     9     10    ...  \
id_zona                                                              ...   
1         0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0   8.0   9.0  ...   
2         1.0   0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0   8.0  ...   
3         2.0   1.0   0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0  ...   
4         3.0   2.0   1.0   0.0   1.0   2.0   3.0   4.0   5.0   6.0  ...   
5         4.0   3.0   2.0   1.0   0.0   1.0   2.0   3.0   4.0   5.0  ...   

id_zona       2091       2092       2093       2094       2095       2096  \
id_zona                                                                     
1        71.840100  72.124892  72.422372  72.732386  73.054774  73.389373   
2        71.568149  71.840100  72.124892  72.422372  72.732386  73.054774   
3        71.309186  71.568149  71.840100  72.124892  72.422372  72.732386   
4        71.063352  71.309186  71.568149  71.840100  72.124892  72.422372   
5    

Una primera visualización del metodo Clark-Wright:

In [8]:
from itertools import combinations

In [29]:
path_venta_zona_1 = os.path.join('..', '..', 'Datos', 'venta_zona_1_20250115.csv')
clientes_1_data = pd.read_csv(path_venta_zona_1)
path_flota = os.path.join('..', '..', 'Datos', 'flota_20250115.csv')
flota_data = pd.read_csv(path_flota)
path_camiones = os.path.join('..', '..', 'Datos', 'vehiculos_20250115.csv')
camiones_data = pd.read_csv(path_camiones)

print("Datos de clientes:")
print(clientes_1_data.head())
print("Datos de flota:")   
print(flota_data.head())    
print("Datos de camiones:")
print(camiones_data.head())

# === Calcular demanda por zona ===
demanda_por_zona = clientes_1_data.groupby('id_zona')['venta_digital'].sum().reset_index()
zonas_datos = pd.merge(demanda_por_zona, zonas_data, on='id_zona')

print("Datos de zonas con demanda:")    
print(zonas_datos.head())
  
print("Datos de tiendas:")
for index, row in tiendas_data.iterrows():
    print(f"Fila {index}:")
    print(row)
    print("-" * 40)  # Separador para mayor claridad

# === Agrupar por tienda física ===
tiendas = zonas_datos['tienda_zona'].unique()
rutas_totales = {}

for index, row in tiendas_data.iterrows():
    tienda = row['id_tienda']
    print(f"\nProcesando tienda: {tienda}")

    # Subconjunto de zonas asociadas a esta tienda
    sub_zonas = zonas_datos[zonas_datos['tienda_zona'] == tienda].copy()
    sub_zonas = sub_zonas.reset_index(drop=True)

    # Obtener tipo y cantidad de camiones para la tienda
    flota_info = flota_data[flota_data['id_tienda'] == tienda]
    if flota_info.empty:
        print(f"No hay datos de flota para tienda {tienda}, se omite.")
        continue

    # flota_info tiene solo una fila
    id_camion = flota_info.iloc[0]['id_camion']
    n_camiones = flota_info.iloc[0]['N']
    capacidad = camiones_data.loc[camiones_data['tipo_camion'] == id_camion, 'Q'].values[0]

    # Establecer el depósito como primer punto
    #deposito_coord = sub_zonas.iloc[0][['x_zona', 'y_zona']].values
    
    deposito_coord = row[['pos_x', 'pos_y']].values.astype(float)
    
    # Restar 1 a cada coordenada para que sea 0-indexed
    deposito_coord = [coord - 1 for coord in deposito_coord]

    print(f"Coordenadas del depósito: {deposito_coord}")
    sub_zonas['nodo'] = range(1, len(sub_zonas) + 1)
    zona_id_map = {row['nodo']: row['id_zona'] for _, row in sub_zonas.iterrows()}
    #print(f"Zona ID map: {zona_id_map}")

    deposito_fila = sub_zonas[
        (sub_zonas['x_zona'] == deposito_coord[0]) & (sub_zonas['y_zona'] == deposito_coord[1])
    ]
    print(f"Deposito fila: {deposito_fila}")

    if not deposito_fila.empty:
        # Obtener la demanda del depósito desde sub_zonas
        demanda_deposito = deposito_fila.iloc[0]['venta_digital']
        # Eliminar la fila del depósito de sub_zonas para evitar duplicados
        sub_zonas = sub_zonas.drop(deposito_fila.index)
    else:
        # Si no se encuentra, manejar el caso (puedes lanzar un error o asignar valores predeterminados)
        raise ValueError("El depósito no está en sub_zonas.")

    coords = np.vstack([deposito_coord, sub_zonas[['x_zona', 'y_zona']].values])

    # Revisar el cero, la zona de la tienda si puede tener demanda creo
    demanda = np.concatenate([[demanda_deposito], sub_zonas['venta_digital'].values])
    dist = cdist(coords, coords)

    #print(f"coords: {coords}")

    rutas = {i: [0, i, 0] for i in range(1, len(coords))}
    savings = []

    for i, j in combinations(range(1, len(coords)), 2):
        s = dist[0, i] + dist[0, j] - dist[i, j]
        savings.append((s, i, j))
    savings.sort(reverse=True)

    for s, i, j in savings:
        ruta_i = next((r for r in rutas.values() if i in r[1:-1]), None)
        ruta_j = next((r for r in rutas.values() if j in r[1:-1]), None)

        if ruta_i is None or ruta_j is None or ruta_i == ruta_j:
            continue

        carga_i = sum(demanda[k] for k in ruta_i if k != 0)
        carga_j = sum(demanda[k] for k in ruta_j if k != 0)
        if carga_i + carga_j > capacidad:
            continue

        if ruta_i[-2] == i and ruta_j[1] == j:
            nueva_ruta = ruta_i[:-1] + ruta_j[1:]
        elif ruta_j[-2] == j and ruta_i[1] == i:
            nueva_ruta = ruta_j[:-1] + ruta_i[1:]
        else:
            continue

        rutas = {k: v for k, v in rutas.items() if v != ruta_i and v != ruta_j}
        rutas[i] = nueva_ruta

    # === Seleccionar como máximo N rutas con menor distancia total ===
    rutas_finales = []
    for ruta in rutas.values():
        zonas_ruta = [zona_id_map[i] if i != 0 else f"DEPOSITO_{tienda}" for i in ruta]
        carga = sum(demanda[i] for i in ruta if i != 0)
        distancia = sum(dist[ruta[k]][ruta[k+1]] for k in range(len(ruta)-1))
        rutas_finales.append({
            'ruta': zonas_ruta,
            'carga': carga,
            'distancia': round(distancia, 2)
        })

    rutas_finales.sort(key=lambda r: r['distancia'])  # priorizar rutas cortas
    rutas_totales[tienda] = rutas_finales[:int(n_camiones)]

# === Mostrar resultados ===
for tienda, rutas in rutas_totales.items():
    print(f"\n Rutas desde tienda: {tienda}")
    for i, r in enumerate(rutas):
        print(f"  Ruta {i+1}: {r['ruta']}, Carga: {r['carga']}, Distancia: {r['distancia']}")

Datos de clientes:
   Unnamed: 0  id_zona  id_producto  venta_digital
0           0        1            1              1
1           1        1            2             23
2           2        1            3             37
3           3        1            4             18
4           4        1            5              1
Datos de flota:
   Unnamed: 0  id_tienda  id_camion  N
0           0          1          3  4
1           1          2          3  1
2           2          3          3  3
3           3          4          3  4
4           4          5          2  5
Datos de camiones:
   Unnamed: 0  tipo_camion          Q
0           0            1  280000000
1           1            2  140000000
2           2            3   80000000
Datos de zonas con demanda:
   id_zona  venta_digital  Unnamed: 0  x_zona  y_zona  tienda_zona
0        1          23631           0       0       0            6
1        2          29445           1       0       1            6
2        3          30804